In [14]:
# Cell 1: Import tất cả các thư viện cần thiết
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import warnings
import sqlite3
from datetime import datetime

warnings.filterwarnings('ignore')

In [15]:
# Cell 2: Định nghĩa hàm reduce_mem_usage
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics: 
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [16]:
# Cell 3: Đọc dữ liệu
def read_data():
    INPUT_DIR_PATH = 'C:/Users/Ho Hau/Downloads/M5/data/raw/'
    sell_prices_df = pd.read_csv(INPUT_DIR_PATH + 'sell_prices.csv')
    sell_prices_df = reduce_mem_usage(sell_prices_df)
    print('Sell prices has {} rows and {} columns'.format(sell_prices_df.shape[0], sell_prices_df.shape[1]))
    
    calendar_df = pd.read_csv(INPUT_DIR_PATH + 'calendar.csv')
    calendar_df = reduce_mem_usage(calendar_df)
    print('Calendar has {} rows and {} columns'.format(calendar_df.shape[0], calendar_df.shape[1]))
    
    sales_train_validation_df = pd.read_csv(INPUT_DIR_PATH + 'sales_train_validation.csv')
    sales_train_validation_df = reduce_mem_usage(sales_train_validation_df)
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation_df.shape[0], sales_train_validation_df.shape[1]))
    
    return sell_prices_df, calendar_df, sales_train_validation_df

sell_prices_df, calendar_df, sales_train_validation_df = read_data()

Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 95.00 Mb (78.7% reduction)
Sales train validation has 30490 rows and 1919 columns


In [17]:
# Kiểm tra các cột trong calendar_df
print("Columns in calendar_df:", calendar_df.columns.tolist())
if 'date' not in calendar_df.columns:
    raise KeyError("Column 'date' not found in calendar_df. Please check the data.")

Columns in calendar_df: ['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']


In [18]:
# Cell 4: Chuyển dữ liệu sang dạng long
d_cols = [c for c in sales_train_validation_df.columns if c.startswith('d_')]
d_cols = d_cols[-360:]  # Lấy 360 ngày cuối
sales_long = sales_train_validation_df.melt(id_vars=['item_id', 'store_id'], value_vars=d_cols, var_name='d', value_name='sales')


In [19]:
# Gộp với calendar_df để lấy các cột thời gian và sự kiện
calendar_df['date'] = pd.to_datetime(calendar_df['date'])
sales_long = sales_long.merge(calendar_df[['d', 'date', 'wm_yr_wk', 'event_name_1', 'event_name_2', 'snap_CA', 'snap_TX', 'snap_WI']], on='d')
# Kiểm tra các cột trong sales_long sau khi merge với calendar_df
print("Columns in sales_long after merging with calendar_df:", sales_long.columns.tolist())
if 'date' not in sales_long.columns:
    raise KeyError("Column 'date' not found in sales_long after merging with calendar_df.")
# Gộp với sell_prices_df thông qua wm_yr_wk
sales_long = sales_long.merge(sell_prices_df, on=['store_id', 'item_id', 'wm_yr_wk'], how='left')

Columns in sales_long after merging with calendar_df: ['item_id', 'store_id', 'd', 'sales', 'date', 'wm_yr_wk', 'event_name_1', 'event_name_2', 'snap_CA', 'snap_TX', 'snap_WI']


In [20]:
# Hàm chọn mã đại diện
def select_representative_items(df, stores, categories=['FOODS', 'HOUSEHOLD']):
    selected_items = []
    for store in stores:
        for category in categories:
            # Lọc dữ liệu của cửa hàng và danh mục
            df_store = df[df['store_id'] == store]
            df_store = df_store[df_store['item_id'].str.startswith(category)]
            
            # Tính tổng doanh số cho mỗi sản phẩm
            item_sales = df_store.groupby('item_id')['sales'].sum().reset_index()
            
            if not item_sales.empty:
                # Sắp xếp theo doanh số
                item_sales = item_sales.sort_values('sales')
                # Chọn sản phẩm có doanh số thấp nhất
                if len(item_sales) > 0:
                    selected_items.append(item_sales.iloc[0]['item_id'])
                # Chọn sản phẩm có doanh số trung bình (gần trung vị)
                if len(item_sales) > 1:
                    median_sales = item_sales['sales'].median()
                    closest_item = item_sales.iloc[(item_sales['sales'] - median_sales).abs().argmin()]
                    selected_items.append(closest_item['item_id'])
                # Chọn sản phẩm có doanh số cao nhất
                if len(item_sales) > 2:
                    selected_items.append(item_sales.iloc[-1]['item_id'])
    
    # Lọc dữ liệu chỉ giữ các sản phẩm đại diện
    return df[df['item_id'].isin(selected_items)]

In [21]:
# Chọn mã đại diện cho tất cả các cửa hàng
all_stores = ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']
df = select_representative_items(sales_long, all_stores)

In [28]:
import pickle
# Hàm tạo đặc trưng
def add_features(df):
    # Đảm bảo cột date tồn tại và ở định dạng datetime
    if 'date' not in df.columns:
        raise KeyError("Column 'date' not found in df before applying add_features.")
    df['date'] = pd.to_datetime(df['date'])
    df['day'] = df['date'].dt.day
    df['month'] = df['date'].dt.month
    df['weekday'] = df['date'].dt.weekday
    df['week'] = df['date'].dt.isocalendar().week

    # Chuẩn hóa sự kiện và SNAP
    df['event'] = ((df['event_name_1'].notnull()) | (df['event_name_2'].notnull())).astype(int)
    df['snap'] = df.apply(lambda x: 1 if (x['store_id'].startswith('CA') and x['snap_CA'] == 1) or 
                                        (x['store_id'].startswith('TX') and x['snap_TX'] == 1) or 
                                        (x['store_id'].startswith('WI') and x['snap_WI'] == 1) else 0, axis=1)

    # Đặc trưng chu kỳ
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 7)
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 7)
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

    # Lag và rolling mean
    for lag in [7, 14, 28]:
        df[f'sales_lag_{lag}'] = df.groupby(['item_id', 'store_id'])['sales'].shift(lag)
    df['rolling_mean_7'] = df.groupby(['item_id', 'store_id'])['sales'].transform(lambda x: x.shift(1).rolling(7).mean())
    df['rolling_mean_14'] = df.groupby(['item_id', 'store_id'])['sales'].transform(lambda x: x.shift(1).rolling(14).mean())

    # Điền NaN
    df.fillna({'sell_price': 0, 'sales': 0, 'sales_lag_7': 0, 'sales_lag_14': 0, 'sales_lag_28': 0, 'rolling_mean_7': 0, 'rolling_mean_14': 0}, inplace=True)

    # Chuẩn hóa đặc trưng số
    scaler = MinMaxScaler()
    numeric_cols = ['sales', 'sell_price', 'sales_lag_7', 'sales_lag_14', 'sales_lag_28', 'rolling_mean_7', 'rolling_mean_14']
    df[numeric_cols] = scaler.fit_transform(df[numeric_cols])
    with open('utils/scaler.pkl', 'wb') as f:
        pickle.dump(scaler, f)
    # Mã hóa lại item_id và store_id SAU KHI lọc dữ liệu
    item_enc = LabelEncoder()
    store_enc = LabelEncoder()
    df['item_idx'] = item_enc.fit_transform(df['item_id'])
    df['store_idx'] = store_enc.fit_transform(df['store_id'])

    return df, item_enc, store_enc

In [23]:
# Áp dụng cho dữ liệu gộp
df, item_enc, store_enc = add_features(df)

In [24]:
# Lưu dữ liệu vào SQLite
conn = sqlite3.connect('processed_data.db')
df.drop(['item_id', 'store_id', 'event_name_1', 'event_name_2', 'snap_CA', 'snap_TX', 'snap_WI', 'weekday', 'month'], axis=1, inplace=True)
df.to_sql('data', conn, if_exists='replace', index=False)
conn.close()

In [25]:
# Lưu bộ mã hóa
import pickle
with open('utils/item_encoder.pkl', 'wb') as f:
    pickle.dump({item: idx for idx, item in enumerate(item_enc.classes_)}, f)
with open('utils/store_encoder.pkl', 'wb') as f:
    pickle.dump({store: idx for idx, store in enumerate(store_enc.classes_)}, f)

# Lưu thông tin đặc trưng
feature_cols = ['day', 'weekday_sin', 'weekday_cos', 'month_sin', 'month_cos', 'week',
                'sales_lag_7', 'sales_lag_14','sales_lag_28', 'rolling_mean_7','rolling_mean_7', 'sell_price', 'event', 'snap']

In [26]:
# Kiểm tra item_enc và store_enc
if 'item_enc' not in globals() or 'store_enc' not in globals():
    raise NameError("item_enc or store_enc is not defined. Ensure add_features() returned these variables correctly.")

try:
    with open('model_params.txt', 'w') as f:
        f.write(f"num_items: {len(item_enc.classes_)}\n")
        f.write(f"num_stores: {len(store_enc.classes_)}\n")
        f.write(f"feature_cols: {feature_cols}\n")
except PermissionError as e:
    print(f"PermissionError: Unable to write to 'model_params.txt'. Check file permissions or if the file is in use: {e}")
except Exception as e:
    print(f"Error while writing to 'model_params.txt': {e}")